# 多任务之异步协同
## 两种程序
- CPU密集型：连续占用CPU进行密集的算术逻辑运算；
- I/O密集型：有断续的I/O访问；
- 前提事实：I/O访问等待时间（毫秒）比CPU计算时间（纳秒）高几个数量级。

```python
def f(name, n):  # 一个普通的函数
    print(name, strftime("%X"), "START", n)
    for i in range(n):
        sleep(1)
    print(name, strftime("%X"), "END")
    
async def fcpu(name, n):  # 异步函数（模拟CPU密集型）
    print(name, strftime("%X"), "START", n)
    for i in range(n):
        sleep(1)  # 普通的sleep，啥都不干，但要占着CPU
    print(name, strftime("%X"), "END")
    
async def fio(name, n):  # 异步函数（模拟IO密集型）
    print(name, strftime("%X"), "START", n)
    for i in range(n):
        await asyncio.sleep(1)  # 异步sleep，啥都不干，可以让出CPU
    print(name, strftime("%X"), "END")
```

![](funcdef.png)

## 两种执行方式
- 同步执行：一个函数执行完成之后，再执行另一个函数；
- 异步执行：**不必**等一个函数执行完成，只要有CPU空闲让出，就可以启动别的函数执行；
- 异步执行什么时候能够提高程序性能？
  - 前提事实：I/O访问等待时间（毫秒）比CPU计算时间（纳秒）高几个数量级
  - 在I/O访问等待开始的时候，让出CPU时间

![](timecpu.png)

![](timeio.png)

```python
async def main():
    print(f"started main at {strftime('%X')}")
    async with asyncio.TaskGroup() as tg:
        for i in range(5):
            tg.create_task(fio(f"f{i:02d}", randint(1, 5)))
    print(f"finished main at {strftime('%X')}")
    
asyncio.run(main())
```

## 异步执行CPU密集型程序？
- 用异步多线程可以提高性能！

```python
async def main():
    print(f"started main at {strftime('%X')}")
    async with asyncio.TaskGroup() as tg:
        for i in range(5):
            tg.create_task(
                asyncio.to_thread(f, f"f{i:02d}", randint(1, 5))
                )
    print(f"finished main at {strftime('%X')}")
    
asyncio.run(main())
```

![](timethread.png)